# Stage 13 Homework Starter — Productization
Sparsh Patel 08/28/2025
## Objective
Deploy the trained model as a **reusable, handoff-ready API or dashboard** and finalize your project for reproducibility and clarity.

In [7]:
import os
os.chdir('/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework13')
print(os.getcwd())

/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework13


In [3]:
pip install utils

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13983 sha256=8d7966cd72a60f4f9cd5ffabd3e910092fdb785d24fb1a56d75280f94ec20ea0
  Stored in directory: /Users/sparshpatel/Library/Caches/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. Create mock, very basic analysis

In [10]:
sys.path.append('/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework13/src')
from src.utils import validate_inputs


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import pickle
import os
import sys


sys.path.append('/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework13/src')
from src.utils import validate_inputs, calculate_confidence_interval, preprocess_features
data_path = '/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework13/data/h13data.csv'
df = pd.read_csv(data_path)

df['moneyness'] = df['underlying_price'] / df['strike']
df['vol_time'] = df['implied_volatility'] * df['time_to_expiry']

features = ['implied_volatility', 'moneyness', 'vol_time']
target = 'market_price'
clean_df = df[features + [target]].dropna()

X = clean_df[features]
y = clean_df[target]

print(f"Final dataset: {X.shape[0]} samples, {X.shape[1]} features")
print(f"Target range: ${y.min():.2f} - ${y.max():.2f}")
print("Basic analysis complete.")

Final dataset: 496 samples, 3 features
Target range: $0.01 - $495.83
Basic analysis complete.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:")
print(f"MAE: ${mae:.2f}")
print(f"R²: {r2:.3f}")
print(f"Features: {features}")

test_vol, test_moneyness, test_vol_time = 0.25, 1.05, 0.05
validation_errors = validate_inputs(test_vol, test_moneyness, test_vol_time)
if validation_errors:
    print(f"Validation errors: {validation_errors}")
else:
    print("✓ Test inputs passed validation")

Model Performance:
MAE: $66.84
R²: 0.344
Features: ['implied_volatility', 'moneyness', 'vol_time']
✓ Test inputs passed validation


## 5. Pickle / Save Final Model



In [ ]:
os.makedirs('/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework13/model', exist_ok=True)


model_data = {
    'model': model,
    'features': features,
    'mae': mae,
    'r2': r2,
    'n_samples': len(X)
}

with open('/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework13/model/hw13model.pkl', 'wb') as f:
    pickle.dump(model_data, f)


with open('/Users/sparshpatel/bootcamp_sparsh_patel/homework/homework13/model/hw13model.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
    loaded_model = loaded_data['model']


test_features = [0.25, 1.05, 0.05]  
test_pred = loaded_model.predict([test_features])[0]


confidence_result = calculate_confidence_interval(test_pred, mae)

print(f"Test prediction: ${test_pred:.2f}")
print(f"95% CI: ${confidence_result['lower_bound']:.2f} - ${confidence_result['upper_bound']:.2f}")
print(f"Model loaded successfully with {len(loaded_data['features'])} features")

Test prediction: $33.39
95% CI: $-97.61 - $164.40
Model loaded successfully with 3 features


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


## 6. Flask API Starter

 Implement Flask endpoints for /predict and /plot

In [ ]:
import requests
import time


test_cases = [
    (0.25, 1.05, 0.05),  
    (0.25, 1.05, 0.6),   
    (3.0, 1.05, 0.05),   
    (0.25, 0.3, 0.05),   
]

print("Testing input validation with utils:")
for vol, money, vol_time in test_cases:
    errors = validate_inputs(vol, money, vol_time)
    status = "✓ VALID" if not errors else f"✗ INVALID: {errors[0]}"
    print(f"Vol={vol}, Money={money}, VolTime={vol_time} → {status}")

print("\nFlask API Testing:")
print("Start the API with: python app.py")


try:

    response = requests.get('http://127.0.0.1:5000/predict/0.25/1.05/0.05')
    print(f"✓ API prediction: {response.json()}")
    

    response = requests.get('http://127.0.0.1:5000/predict/3.0/1.05/0.05')
    print(f"✗ API validation: {response.json()}")
    

    test_data = {'implied_volatility': 0.25, 'moneyness': 1.05, 'vol_time': 0.05}
    response = requests.post('http://127.0.0.1:5000/predict', json=test_data)
    print(f"✓ POST prediction: {response.json()}")
except Exception as e:
    print(f"Cheking API  {e}")